In [1]:
pip install visen

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for bogo: filename=bogo-1.1-cp36-none-any.whl size=28135 sha256=93e2a48d31f6916a87e7773cd3ee8953c684efc4d257a83714c0cce02bb60a58
  Stored in directory: /root/.cache/pip/wheels/5d/90/02/49165c6c062e760067fa27ced64563f228acd9ee2be501d4eb
Successfully built bogo


In [2]:
pip install fasttext

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3018759 sha256=e7f2fb391bc51d1327392d13dc7d777d9c9436a64c16469d93463a283dccaa9f
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [3]:
pip install pyvi

     |████████████████████████████████| 8.5MB 2.8MB/s 
     |████████████████████████████████| 747kB 43.0MB/s 


In [4]:
import pandas as pd
import re
import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from gensim.models import FastText
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Dropout, Bidirectional, Conv1D,MaxPooling1D,Flatten, SpatialDropout1D,Concatenate
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU,LSTM
from gensim.models import Word2Vec
from sklearn.metrics import precision_recall_fscore_support as score
import fasttext
import time
import visen
from pyvi import ViTokenizer, ViPosTagger

Using TensorFlow backend.


In [5]:
colnames = ['id', 'free_text']
data1 = pd.read_csv('/content/drive/My Drive/SentimentAnalysis/data/02_train_text.csv', names=colnames)

colnames2 = ['id', 'label_id']
data2 = pd.read_csv('/content/drive/My Drive/SentimentAnalysis/data/03_train_label.csv')
df = pd.merge(data1, data2, on="id", how="inner")
del df['id']
data3 = pd.read_csv('/content/drive/My Drive/SentimentAnalysis/data/06_test_text_label.csv')
label_test = data3.label.to_list()
text_test = data3.free_text.to_list()

In [6]:
# Divide dataset to 8/2 train, test for label 0,1,2
Df1 = pd.DataFrame()
Df1['text_0'] = df['free_text'].groupby(df['label_id']).apply(list)[0]
np.shape(Df1['text_0'])
Df1['label_0'] = np.zeros((18614,), dtype=np.int32)

Df2 = pd.DataFrame()
Df2['text_1'] = df['free_text'].groupby(df['label_id']).apply(list)[1]
np.shape(Df2['text_1'])
Df2['label_1'] = np.ones((1022,), dtype=np.int32)

Df3 = pd.DataFrame()
Df3['text_2'] = df['free_text'].groupby(df['label_id']).apply(list)[2]
Df3['label_2'] = np.full(shape=709, fill_value=2, dtype=np.int32)

In [7]:
from sklearn.model_selection import train_test_split

x_data_0 = Df1['text_0']
y_data_0 = Df1['label_0']
x_train_0, x_test_0, y_train_0, y_test_0 = train_test_split(np.array(x_data_0), np.array(y_data_0), test_size=0.2,
                                                            shuffle=False)

x_data_1 = Df2['text_1']
y_data_1 = Df2['label_1']
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(np.array(x_data_1), np.array(y_data_1), test_size=0.2,
                                                            shuffle=False)

x_data_2 = Df3['text_2']
y_data_2 = Df3['label_2']
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(np.array(x_data_2), np.array(y_data_2), test_size=0.2,
                                                            shuffle=False)

In [8]:
X_train = np.concatenate((x_train_0, x_train_1, x_train_2), axis=None)
X_test = np.concatenate((x_test_0, x_test_1, x_test_2), axis=None)
Y_test = np.concatenate((y_test_0, y_test_1, y_test_2), axis=None)
Y_train = np.concatenate((y_train_0, y_train_1, y_train_2), axis=None)



In [31]:
x_data_1 = [each_string.lower() for each_string in x_data_1]
X_data_1_1= normalize_text(x_data_1)
X_data_1_2 = cleanup_text(X_data_1_1)
X_data_1_2[101:500]

['ngoài đời xấu vl',
 'đừng lấy tg đéo nào lấy tao nhá',
 'ụa gì nhạt vl',
 'nếu có người tốt hơn sẽ không níu kéo đâu còn người không tốt hơn thì t đéo buông tha cho m đâu bỏ t đi là m thiệt thòi đấy không ai tốt như t đâu nhé trang thảo shared a post feeling happy with thảo trang url bạn thân nhớ nói nhé oke tao ổn',
 'mỳ tôm chất cả đống ntn thì sợ đéo j chết đói url',
 'page ơi là page vớ vẩm vl',
 'có nhiều bạn thật lạ lùng cứ ngồi chê bai nghề người khác làm rồi mặc định mình là dân cao cấp kiếm tiền chính đáng thì làm gì phải xấu hổ nhà bạn giàu thì bạn hưởng nó đi mà thậm chí nó cũng là của bố mẹ bạn làm nên có phải bạn làm ra đâu mà nói thật chưa chắc đó là tài sản lương thiện nữa kìa thế bạn tự hào về cái gì muốn chê bai theo kiểu mẹ thiên hạ thì ít nhất bạn cũng đẻ ra được thiên hạ nha nhiều bạn hay chê mà đéo bao giờ nhìn lại mình haiz nhìn cứ y như những bộ phận sinh dục biết nói em ghét nhất là thói xem thường người khác mà éo biết mình chẳng hơn ai url',
 'làm ăn lãi vl'

In [9]:
Y_train_label = np.concatenate((Y_train, Y_test), axis=None)


In [10]:
X_train = [each_string.lower() for each_string in X_train]
X_test = [each_string.lower() for each_string in X_test]
text_test = [each_string.lower() for each_string in text_test]

abbrevs = {r'\bk\b': 'không', r'\bko\b': 'không',r"\bkh\b": 'không', r"\b's\b": ' ', r"\be\b": 'em', r"\bsđt\b": 'điện thoại',
           r"\bib\b": 'inbox', r"\bctv\b": 'cộng tác viên',r"\bđkm\b": 'dm',r"\bvcl\b": 'vl',r"\bvclin\b": 'vl',r"\bđm\b": 'dm',r"\bvkl\b": 'vl',r"\bd.m\b": 'dm',r"\bdmm\b": 'dm',r"\bđcm\b": 'dm',,r"\bđmm\b": 'dm',
           r"\bhnoi\b": 'hà nội',r"\bhanoi\b": 'hà nội',r"\bfb\b": 'facebook',r"\bcmt\b": 'comment',r"\bcoment\b": 'comment',r"\bstt\b": 'status'}


def replace_all(text, dic):
    for i, j in dic.items():
        text = re.sub(i, j, text)
    return text


def normalize_text(sentences):
    normalize = []
    for sent in sentences:
        normalize.append(replace_all(sent, abbrevs))
    return normalize


first_clean = normalize_text(X_train)
first_clean_test = normalize_text(X_test)
first_clean_text_test = normalize_text(text_test)


def cleanup_text(sentences):
    cleaned_text = []
    for sent in sentences:
        # get hastag
        sent = re.sub(r'(?:^|\s)(\#\w+)', ' hastag', sent)
        # get email
        sent = re.sub(r'[\w\.-]+@[\w\.-]+', 'email', sent)
        # get url
        sent = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*',
                      ' url', sent)
        # dupplicate word
        # sent = re.sub(r'(\w)\1+', r'\1', sent)
        # delete string that contain number
        sent = re.sub(r"\w*\d\w*", ' ', sent)

        # y = re.sub(r'\b\d+([\.,]\d+)?', ' number', g)
        # delete number in string
        # g = re.sub(r"\d*([^\d\W]+)\d*", r'\1', z)
        sent = re.sub('[^\w ]', ' ', sent)
        sent = visen.clean_tone(sent)
        cleaned_text.append(sent)
    return cleaned_text


second_clean = cleanup_text(first_clean)
second_clean_test = cleanup_text(first_clean_test)
second_clean_text_test = cleanup_text(first_clean_text_test)


# tokenizer for sentencse
def tokenize_sentences(sentences):
    tokens_list = []
    for sent in sentences:
        tokens = ViTokenizer.tokenize(sent)
        tokens_list.append(tokens)
    return tokens_list


third_clean = tokenize_sentences(second_clean)
third_clean_test = tokenize_sentences(second_clean_test)
third_clean_text_test = tokenize_sentences(second_clean_text_test)


In [11]:
with open("/content/drive/My Drive/SentimentAnalysis/vietnamese-stopwords.txt", encoding="utf8") as f:
    stop_word = f.readlines()
stop_word = [x.strip() for x in stop_word]


def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split()
                      if word not in stop_word])
        )

    return removed_stop_words
remove_stop = remove_stop_words(third_clean)
remove_stop_test = remove_stop_words(third_clean_test)

In [12]:
def get_train_input(train_texts):
    train_input = []
    model = fasttext.load_model("/content/drive/My Drive/SentimentAnalysis/models/cc.vi.300.bin")
    for line in train_texts:
        embedding = []
        for x in range(100):
            if len(line) <= x:
                embedding.append(np.zeros(300, dtype=np.float32))
            else:
                # try:
                c = model.get_word_vector(line[x])
                # except KeyError:
                #     c = np.zeros(80, dtype=np.float32)
                embedding.append(c)
        train_input.append(embedding)
    return train_input

In [13]:
def split_word(sentences):
    new_arr = []
    for sent in sentences:
        new_arr.append(sent.split())
    return new_arr


fourth_clean = split_word(remove_stop)
fourth_clean_test = split_word(remove_stop_test)
# fourth_clean_train = np.concatenate((fourth_clean, fourth_clean_test), axis=None)
# fourth_clean_text_test = tokenize_sentences(third_clean_text_test)


In [14]:
start_time = time.time()
# X_text_train = np.array(get_train_input(fourth_clean_train))
# X_text_test = np.array(get_train_input(fourth_clean_text_test))
# X_label = tf.keras.utils.to_categorical(Y_train_label, num_classes=3)
X_text_train = np.array(get_train_input(fourth_clean))
X_text_test= np.array(get_train_input(fourth_clean_test))
X_label_train = tf.keras.utils.to_categorical(Y_train, num_classes=3)

input_dim = (X_text_train.shape[1], X_text_train.shape[2])

from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [32]:
# build model
model = Sequential()
model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_dim))
model.add(Dropout(0.2))
# model.add(Bidirectional(GRU(32)))
model.add(Conv1D(filters=100, kernel_size=3, activation='relu', input_shape=input_dim))
# model.add(Dropout(0.2))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])
model.summary()
history = model.fit(X_text_train, X_label_train, epochs=10, batch_size=64,verbose=1)

print()
print("Execution Time %s seconds: " % (time.time() - start_time))
from sklearn.metrics import classification_report

start_time_test = time.time()
prediction = model.predict(X_text_test)
label = []
for n in prediction:
    label.append(np.argmax(n))
# print(prediction)

print(classification_report(Y_test, label))
print()
print("Execution Time test %s seconds: " % (time.time() - start_time_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 100, 256)          329472    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 98, 100)           76900     
_________________________________________________________________
dropout_4 (Dropout)          (None, 98, 100)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 49, 100)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4900)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)              

In [33]:
# text_1=['Quản Tùng Lâm đcm công việc tốt vl =)))','em nói cho chị cổ lực na trát nghe ở bển chị cứ lo mần ăn đừng có hơn thua với mấy con đĩ này làm gì mấy con đĩ này là mấy con đĩ nghèo khổ khi nào chị dề việt nam đi em cho chị một miếng đất em có 7 miếng lận nè 1 miếng nè 2 miếng nè 3 miếng nè 4 miếng nè 5 miếng nè 6 miếng nè 7 miếng nè','Mày làm 1 tháng 10 triệu ở Việt Nam là ổn rồi, còn kêu ca gì nữa. Mày muốn gây bạo động lật đổ chính quyền à ? Thế thì lực lượng Công an ra tay dẹp loạn ngay. Mùa xuân dân chủ của lũ mày trong tù giam chứ không được thảnh thơi đâu. Mày so bì con nhà giàu, bộ mày không muốn dân Việt Nam giàu à hay muốn toàn dân nhà nghèo ?','thằng này óc chó vl','Kệ tao -_- sân si vl','Đề nghị đã đăng bọn rác rưởi phân hôi , cần thiết phải hình ảnh không bị che , như sex JAV','Sống trên đời này thì chỉ có tình thương mến thương . Chứ nói về độ sợ nhau thì thằng cha mày bố mày còn tát tai chứ mày ??  <URL>']
text_test = [each_string.lower() for each_string in text_test]
text_2= normalize_text(text_test)
text_3 = cleanup_text(text_2)
text_4= tokenize_sentences(text_3)
stop_w = remove_stop_words(text_4)
text_5 = split_word(stop_w)
text_6 = np.array(get_train_input(text_5))


In [34]:
x=model.predict(text_6)
y=[]
for n in x:
    y.append(np.argmax(n))
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [35]:
print(classification_report(label_test, y))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      4640
           1       0.29      0.17      0.21       302
           2       0.47      0.28      0.35       144

    accuracy                           0.91      5086
   macro avg       0.57      0.48      0.51      5086
weighted avg       0.89      0.91      0.90      5086



In [ ]:
remove_stop[:14891]